In [21]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import os
import scipy.signal
import fasttext

from numba import prange
from numba import jit
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score,accuracy_score
from tqdm import tqdm
from vmdpy import VMD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE

os.chdir('../')
os.chdir('data/')

In [22]:
def format_text(df,col):
      comp_df = df.copy()
      # remove all the punctuation
      comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

      #Remove all non-character
      comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

      # Remove extra space
      comp_df[col] = comp_df[col].str.strip()

      # Change to lowercase
      comp_df[col] = comp_df[col].str.lower()
      return comp_df

def energy(u):
# Estimate PSD `S_xx_welch` at discrete frequencies `f_welch`
    f_welch, S_xx_welch = scipy.signal.welch(u)
    # Integrate PSD over spectral bandwidth
    # to obtain signal power `P_welch`
    df_welch = f_welch[1] - f_welch[0]
    return np.sum(S_xx_welch) * df_welch  

def maxvdm(f):
    alpha = 6     
    tau = 0            
    K = 2       
    DC = 0             
    init = 1           
    tol = 1e-9
    u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol) 
    energy_array=[]
    for i in u:
        energy_array.append(energy(i))
    ind = np.argmax(energy_array)
    return u[ind]

https://www.kaggle.com/c/nlp-getting-started/overview

In [23]:
tweets = pd.read_csv('train.csv')

In [24]:
train = format_text(tweets,'text')
X = train['text'].tolist()
Y = train['target']

<ipython-input-22-905a7a697882>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')
<ipython-input-22-905a7a697882>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")


In [25]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(X).toarray()

In [26]:
@jit(parallel=True,forceobj=True)
def vm(features):
    X = []
    for i in tqdm(features):
        X.append(maxvdm(i))
    return X

X_vmd = vm(features)

  0%|▎                                                                                                                        | 19/7613 [00:02<11:29, 11.01it/s]c:\users\aniru\appdata\local\programs\python\python39\lib\site-packages\vmdpy\vmdpy.py:105: RuntimeWarning: invalid value encountered in double_scalars
  omega_plus[n+1,k] = np.dot(freqs[T//2:T],(abs(u_hat_plus[n+1, T//2:T, k])**2))/np.sum(abs(u_hat_plus[n+1,T//2:T,k])**2)
c:\users\aniru\appdata\local\programs\python\python39\lib\site-packages\vmdpy\vmdpy.py:114: RuntimeWarning: invalid value encountered in double_scalars
  omega_plus[n+1,k] = np.dot(freqs[T//2:T],(abs(u_hat_plus[n+1, T//2:T, k])**2))/np.sum(abs(u_hat_plus[n+1,T//2:T,k])**2)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7613/7613 [10:29<00:00, 12.10it/s]


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_vmd, Y, test_size=0.25,random_state=22)
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83      1111
           1       0.84      0.58      0.69       793

    accuracy                           0.78      1904
   macro avg       0.80      0.75      0.76      1904
weighted avg       0.79      0.78      0.77      1904



In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_vmd, Y, test_size=0.25,random_state=22)
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83      1111
           1       0.84      0.58      0.69       793

    accuracy                           0.78      1904
   macro avg       0.80      0.75      0.76      1904
weighted avg       0.79      0.78      0.77      1904



In [30]:
# os.chdir('..')
# os.chdir('..')
os.getcwd()

'C:\\Users\\aniru\\Desktop\\Amrita\\Sem 4\\Personal Projects\\VMD on Text Classification'

In [31]:
ft = fasttext.load_model('cc.en.300.bin')
size = ft.get_dimension()
corpus = []
for i in X:
    corpus.append(word_tokenize(i))
X_fast = np.zeros((len(corpus),size)) #Initializing the X matrix with zeros
for i in range(len(corpus)):
    emb = np.zeros((1,size))
    for w in corpus[i]:
        emb = emb +  ft.get_word_vector(w)
    X_fast[i] = emb

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_fast, Y, test_size=0.25,random_state=22)
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83      1111
           1       0.79      0.70      0.74       793

    accuracy                           0.79      1904
   macro avg       0.79      0.78      0.78      1904
weighted avg       0.79      0.79      0.79      1904



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_fast, Y, test_size=0.25,random_state=22)
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.87      0.83      1111
           1       0.79      0.69      0.74       793

    accuracy                           0.79      1904
   macro avg       0.79      0.78      0.78      1904
weighted avg       0.79      0.79      0.79      1904



c:\users\aniru\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
